# ALS


2. Design a workable ALS code. In ALS, please find the best ranks, lambdas, and numIters accordingto RMSE.

In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark\\spark-3.0.1-bin-hadoop2.7'

In [5]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')

In [4]:
import pandas as pd

In [6]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

In [7]:
ratings = pd.read_table("C:/Users/SALONI/Documents/Semester 4/Cloud Computing/Assignment 2/u.data",names = ['UserID', 'MovieID', 'Rating', 'Timestamp'])

In [8]:
ratings.drop('Timestamp', axis=1, inplace=True)

In [9]:
ratings.head()

,UserID,MovieID,Rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [10]:
sqlContext = SQLContext(sc)

In [11]:
Sqldf= sqlContext.createDataFrame(ratings)

In [12]:
#Splitting into train test split - 70-30
(train, test)= Sqldf.randomSplit([0.70,0.30], seed=123)

In [13]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
als = ALS(
         userCol="UserID", 
         itemCol="MovieID",
         ratingCol="Rating", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="drop"
)


In [14]:
type(als)

pyspark.ml.recommendation.ALS

In [15]:
# Adding hyperparameters
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [5,10]) \
            .addGrid(als.regParam, [.01, .03]) \
            .addGrid(als.maxIter,[10,15,20])\
            .build()

In [16]:
# Defining evaluator
evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="Rating", 
           predictionCol="prediction") 
print ("Number of models to be tested: ", len(param_grid))

Number of models to be tested:  12


In [17]:
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

In [18]:
print(cv)

CrossValidator_f319509335ea


In [19]:
#training the model
model=cv.fit(train)

In [20]:
final_model= model.bestModel

In [21]:
type(final_model)

pyspark.ml.recommendation.ALSModel

In [22]:
final_model

ALSModel: uid=ALS_c2e7705fdabc, rank=5

In [27]:
print('Max iteration:', final_model._java_obj.parent().getMaxIter())
print('Rank', final_model._java_obj.parent().getRank())
print('Lambda value: ',final_model._java_obj.parent().getRegParam())


Max iteration: 10
Rank 5
Lambda value:  0.03


In [28]:
test_predictions = final_model.transform(test)

In [29]:
RMSE = evaluator.evaluate(test_predictions)
print('The RMSE Value for ALS is: ',RMSE)

The RMSE Value for ALS is:  0.9547824117167129
